In [3]:
# cleanup blastn of pangenome against the 16 strain ORF files, perc_identity=40
# NOTE: some ORFs are not in frame, that's why using blastn

In [4]:
import pandas as pd
import os
import re
from Bio.Seq import Seq
from Bio import SeqIO

In [5]:
# map length of pangenome ORFs 
input_file = '../data/1011_pangenome/allORFs_pangenome.fasta'
handle = []
seq = []

for record in SeqIO.parse(input_file, "fasta"):
    handle.append(record.id)
    seq.append(str(record.seq))
    
ref_df = pd.DataFrame({'ref_id':handle,'ref_seq':seq})

ref_df['ref_len'] = ref_df.ref_seq.str.len()

ref_map = dict(zip(ref_df.ref_id, ref_df.ref_len))

In [6]:
# combine all the .tsv files into single .tsv
files = [ 
 'pangenome_MSY24.tsv',
 'pangenome_MSY25.tsv',
 'pangenome_MSY26.tsv',
 'pangenome_MSY27.tsv',
 'pangenome_MSY28.tsv',
 'pangenome_MSY29.tsv',
 'pangenome_MSY30.tsv',
 'pangenome_MSY31.tsv',
 'pangenome_MSY32.tsv',
 'pangenome_MSY33.tsv',
 'pangenome_MSY34.tsv',
 'pangenome_MSY35.tsv',
 'pangenome_MSY36.tsv',
 'pangenome_MSY37.tsv',
 'pangenome_MSY38.tsv',
 'pangenome_MSY39.tsv'
]
dataframes = []
blastn_cols = ['query','target','percent_id','alignment_length','mismatch','gapopen','qstart','qend','tstart','tend','evalue','bitscore']

for file in files:
    temp_df = pd.read_csv(f"../data/blastn/{file}", names=blastn_cols, sep='\t')
    dataframes.append(temp_df)
    
df = pd.concat(dataframes)

In [12]:
# map the length of each pangenome ORF
df['q_seq_len'] = df['query'].map(ref_map)

# new column with target strain name
df['t_strain'] = df['target'].str.split('_').str[0]

# percent id whole
df['percent_whole_id'] = (((df['alignment_length']) / df['q_seq_len']) * df['percent_id']/100)

# identify pangenome ORFs that are not found in S288C
df['orf_n'] = df['query'].str.split('-').str[0].astype(int)
df['orf_not_s288c'] = df['orf_n'] <= 1767

df.to_csv('output/blastn_pangenome-to-16strainORFs.csv', index=False)

In [13]:
# get the top percent_id_whole hits
# these are the pangenome hits to each of the 16 strain ORFs
top_idx = df.groupby(['t_strain','query'])['percent_whole_id'].transform(max) == df['percent_whole_id']
top_df = df[top_idx]

top_df = top_df.groupby(['query','t_strain']).percent_whole_id.max().reset_index()
top_df = top_df.pivot(index='query',columns='t_strain',values='percent_whole_id')

top_df.reset_index(inplace=True)

top_df['max_percent'] = top_df.max(axis=1)

top_df['orf_not_s288c'] = top_df['query'].str.split('-').str[0].astype(int) <=1767

top_df.to_csv('output/blastn_pangenome-to-16strainORFs_top-hits.csv', index=False)

/var/folders/fq/q1cflf795cgbnmbhdyn8d9nntyw5jt/T/ipykernel_89177/86104096.py:11: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  top_df['max_percent'] = top_df.max(axis=1)


In [22]:
# identify unique ORFs that did not have a hit, <40% identity threshold

# all pangenome ORFs
pangenome_set = set(ref_map.keys())
# blastn pangenome ORF hits to strains
hit_set = set(top_df['query'].tolist())
# unique pangenoem ORFs list
missing_pangenome = list(pangenome_set - hit_set)

In [23]:
len(missing_pangenome)

0

In [14]:
filename = "pangenome_unique-ORFs.fasta"
f = open(f"output/{filename}", "w")

for index,row in ref_df.query('ref_id.isin(@missing_pangenome)').iterrows():
    f.write(f">{row.ref_id}\n")
    f.write(f"{row.ref_seq}\n")
f.close()

In [ ]:
# blastn 16genomes-to-pangenome
# column names: https://www.metagenomics.wiki/tools/blast/blastn-output-format-6
file = '../data/blastn/16genomes-to-pangenome.tsv'

blastn_cols = ['query','target','percent_id','alignment_length','mismatch','gapopen','qstart','qend','tstart','tend','evalue','bitscore']
df = pd.read_csv(file, names=blastn_cols, sep='\t')



df['tseq_len'] = df.target.map(ref_map)

# percent id whole
df['percent_id_whole'] = (((df.alignment_length) / df.tseq_len) * df.percent_id/100)

df.head()

In [4]:
# blastn 16genomes-to-pangenome
# column names: https://www.metagenomics.wiki/tools/blast/blastn-output-format-6
file = '../data/blastn/pangenome-to-16genomes.tsv'

blastn_cols = ['query','target','percent_id','alignment_length','mismatch','gapopen','qstart','qend','tstart','tend','evalue','bitscore']
df = pd.read_csv(file, names=blastn_cols, sep='\t')

df['qseq_len'] = df['query'].map(ref_map)

# percent id whole
df['percent_whole_id'] = (((df.alignment_length) / df.qseq_len) * df.percent_id/100)

df['orf_n'] = df['query'].str.split('-').str[0].astype(int)

df['orf_not_s288c'] = df['orf_n'] <= 1767

df['t_strain'] = df['target'].str.split('_').str[0]

#df.to_csv('output/pangenome_blastn.csv', index=False)

In [15]:
# orfs in the pangenome missing from the 16 strains
pangenome_set = set(ref_map.keys())

hit_set = set(df['query'].tolist())


missing_pangenome = list(pangenome_set - hit_set)

for index,row in ref_df.query('ref_id.isin(@missing_pangenome)').head().iterrows():
    print(row['ref_id'])

filename = "pangenome_unique.fasta"
f = open(f"output/{filename}", "w")

for index,row in ref_df.query('ref_id.isin(@missing_pangenome)').iterrows():
    f.write(f">{row.ref_id}\n")
    f.write(f"{row.ref_seq}\n")
f.close()

In [21]:
# get the top percent_id_whole
top_idx = df.groupby(['t_strain','query'])['percent_whole_id'].transform(max) == df['percent_whole_id']
top_df = df[top_idx]

top_df = top_df.groupby(['query','t_strain']).percent_whole_id.max().reset_index()
top_df = top_df.pivot(index='query',columns='t_strain',values='percent_whole_id')

top_df.reset_index(inplace=True)

top_df['max_percent'] = top_df.max(axis=1)

top_df['orf_not_s288c'] = top_df['query'].str.split('-').str[0].astype(int) <=1767

top_df.to_csv('output/pangenome_blastn_top-id.csv', index=False)

/tmp/ipykernel_49110/507743706.py:10: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  top_df['max_percent'] = top_df.max(axis=1)


In [74]:
# would need to create a sliding window of 1-100, then just get the raw number of orfs that fall into that category (loop?)
percent_match = []
for i in range(100,0,-1):
    percent_match.append((top_df.max(axis=1) >= i).sum())

/tmp/ipykernel_87177/2384251810.py:4: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  percent_match.append((top_df.max(axis=1) >= i).sum())


In [77]:
top_df

t_strain,query,MSY24,MSY25,MSY26,MSY27,MSY28,MSY29,MSY30,MSY31,MSY32,MSY33,MSY34,MSY35,MSY36,MSY37,MSY38,MSY39,max_percent,orf_not_s288c
0,100-augustus_masked-1375-CFF_4,0.917450,0.916820,0.917450,0.916200,0.916820,0.918070,0.917450,0.916200,0.904940,1.000000,0.916820,0.916820,0.905570,0.916200,0.917450,0.918070,1.000000,True
1,1000-augustus_masked-CGE_5-5812,0.897838,0.897838,0.896792,0.896264,0.896792,0.897310,0.897310,0.896792,0.894680,0.895208,0.895736,0.895208,0.896792,0.896792,0.896792,0.895736,0.897838,True
2,1001-augustus_masked-CGE_5-5812,0.872035,0.871005,0.872035,0.872035,0.872035,0.867907,0.871005,0.871005,0.868936,0.873064,0.869966,0.872035,0.871005,0.872035,0.873064,0.873064,0.873064,True
3,1002-augustus_masked-CGE_5-5812,0.907029,0.903626,0.907029,0.904763,0.907029,0.897957,0.903626,0.904763,0.907029,0.905891,0.904763,0.900223,0.903626,0.905891,0.903626,0.903633,0.907029,True
4,1008-augustus_masked-AID_2-6438,0.900863,0.897989,0.900863,0.897989,0.900863,0.900863,0.896548,0.897989,0.886492,0.897989,0.897989,0.893674,0.899421,0.899421,0.897990,0.900863,0.900863,True
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
7464,995-augustus_masked-AIE_1-5046,0.760028,0.759214,0.760856,0.761670,0.760856,0.760856,0.761670,0.760852,0.758392,0.760856,0.762489,0.760028,0.758396,0.760856,0.762492,0.760028,0.762492,True
7465,996-augustus_masked-AIE_1-5046,0.913180,0.917470,0.913180,0.916400,0.913180,0.913180,0.916400,0.916400,0.916400,0.913180,0.916400,0.911040,0.917470,0.913180,0.912110,0.917470,0.917470,True
7466,997-augustus_masked-AIE_1-5046,0.876164,0.873065,0.876164,0.876164,0.876164,0.876164,0.876164,0.876164,0.874098,0.876164,0.873065,0.877197,0.873065,0.876164,0.877197,0.877197,0.877197,True
7467,998-augustus_masked-BBM_6-7272,0.941600,0.941600,0.941600,0.944440,0.941600,0.935896,0.941600,0.940170,0.943020,0.941600,0.944440,0.941600,0.941600,0.941600,0.940170,0.941600,0.944440,True
